In [191]:
# 기본 패키지 불러오기 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use("seaborn")
sns.set(font_scale=1)
sns.set_style("whitegrid")

# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 맥 OS 폰트 깨짐 방지 코드
from matplotlib import rc
import matplotlib as mpl 
import matplotlib.font_manager as fm 

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 가격 컬럼과의 관계분석을 위한 전처리 진행 (Inside Airbnb: London Data)

### 데이터 불러오기

In [192]:
# 데이터 불러오기
london_lists_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings.csv')
london_lists_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings_outline.csv')
london_neighbourhoods_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/neighbourhoods.csv')
london_reviews_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews.csv')
london_reviews_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews_outline.csv')

# 분석할 df 별도 복사
london_lists = london_lists_raw.copy()
london_reviews = london_reviews_raw.copy()

# 불러온 df 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            90852 non-null  int64  
 1   listing_url                                   90852 non-null  object 
 2   scrape_id                                     90852 non-null  int64  
 3   last_scraped                                  90852 non-null  object 
 4   source                                        90852 non-null  object 
 5   name                                          90852 non-null  object 
 6   description                                   87851 non-null  object 
 7   neighborhood_overview                         47521 non-null  object 
 8   picture_url                                   90842 non-null  object 
 9   host_id                                       90852 non-null 

### 데이터 불러온 최종일자 확인 

In [193]:
london_lists_raw['last_scraped'].unique()

array(['2024-03-21', '2024-03-20', '2024-03-22', '2024-03-19'],
      dtype=object)

슈퍼호스트 선정 기준 날짜 년 4회(분기별)
- 1월 1일 / 4월 1일 / 7월 1일 / 10월 1일 

### 분석에서 사용하지 않을 컬럼 삭제

In [194]:
# 1차 선별한 컬럼 리스트 + 2차 선별 
# 1차 선별한 컬럼 25개 중 21개만 선별 

columns_selected = ['id', 'host_id', 'host_is_superhost', 'neighbourhood_cleansed', \
    'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

# 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'minimum_nights', 'maximum_nights', 'has_availability',

# 선별한 컬럼만 적용
london_lists = london_lists[columns_selected]

# url도 포함한 df (실제 에어비앤비 사이트 들어가서 확인할 용도)
columns_selected_url = ['id', 'listing_url', 'host_id', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', \
    'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

london_url = london_lists_raw[columns_selected_url]

# 변수 정리 
london_lists = london_lists.rename(columns={
    'id': '숙소_id',
    'host_id': '호스트_id',
    # 'host_response_time': '답변_평균시간',
    # 'host_response_rate': '문의_응답률',
    # 'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    # 'host_total_listings_count': '숙소_수',
    # 'host_has_profile_pic' : '프로필_사진',
    # 'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    # 'minimum_nights': '최소_숙박일',
    # 'maximum_nights': '최대_숙박일',
    # 'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
})

# url 포함 df 변수 정리 
london_url = london_url.rename(columns={
    'id': '숙소_id',
    'listing_url' : '숙소_url',
    'host_id': '호스트_id',
    'host_response_time': '답변_평균시간',
    'host_response_rate': '문의_응답률',
    'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'host_has_profile_pic' : '프로필_사진',
    'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
})


# 컬럼명 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        90852 non-null  int64  
 1   호스트_id       90852 non-null  int64  
 2   슈퍼호스트        90463 non-null  object 
 3   숙소_지역        90852 non-null  object 
 4   숙소_유형        90852 non-null  object 
 5   수용_인원수       90852 non-null  int64  
 6   욕실수          57894 non-null  float64
 7   침실수          78118 non-null  float64
 8   침대수          57837 non-null  float64
 9   편의시설         90852 non-null  object 
 10  숙소_가격        57885 non-null  object 
 11  리뷰수          90852 non-null  int64  
 12  30일_리뷰수      90852 non-null  int64  
 13  리뷰점수         65869 non-null  float64
 14  숙소_정확성_리뷰점수  65838 non-null  float64
 15  숙소_청결도_리뷰점수  65843 non-null  float64
 16  숙소_체크인_리뷰점수  65808 non-null  float64
 17  숙소_소통_리뷰점수   65833 non-null  float64
 18  숙소_위치_리뷰점수   65808 non-null  float64
 19  숙소_가

### 결측치 처리에 앞서서 리뷰가 0인 컬럼 제거

In [195]:
# 리뷰수가 0인 컬럼 조건 
condition_review_0 = london_lists['리뷰수'] == 0

# 별도의 DF로 저장
london_lists_review_0 = london_lists[condition_review_0]

리뷰가 0인 데이터의 특징 파악

In [196]:
# 리뷰수가 0인 컬럼의 특징 파악_리뷰데이터 관련

# 개수: 24,983개
print('리뷰가 0개인 런던 내 숙소 개수: ')
print(london_lists_review_0.shape)
print()

# 리뷰가 0개인데 각 리뷰점수가 Null값이 아닌 숙소 개수 

score_type = ['리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수', '평균_리뷰수']

for type in score_type:
    print(f'{type} :')
    print(f'{type}가 null이 아닌 데이터의 개수 >> {london_lists_review_0[type].notnull().sum()}개')
    print()

리뷰가 0개인 런던 내 숙소 개수: 
(24983, 21)

리뷰점수 :
리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_정확성_리뷰점수 :
숙소_정확성_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_청결도_리뷰점수 :
숙소_청결도_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_체크인_리뷰점수 :
숙소_체크인_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_소통_리뷰점수 :
숙소_소통_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_위치_리뷰점수 :
숙소_위치_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_가격_리뷰점수 :
숙소_가격_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

평균_리뷰수 :
평균_리뷰수가 null이 아닌 데이터의 개수 >> 0개



In [197]:
# # 리뷰수가 0인 컬럼의 특징 파악_슈퍼호스트 관련

print(london_lists_review_0['슈퍼호스트'].unique())
print()
print('리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: ')
print(london_lists_review_0['슈퍼호스트'].value_counts())
print()
print('리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:')
print(f'{london_lists_review_0["슈퍼호스트"].isnull().sum()}개')

['f' 't' nan]

리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: 
f    23754
t     1171
Name: 슈퍼호스트, dtype: int64

리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:
58개


리뷰가 0개가 아닌 데이터만 사용

In [198]:
# 리뷰 개수가 0이 아닌 컬럼 조건 설정
condition_review = london_lists['리뷰수'] != 0

# 리뷰 개수가 0이 아닌 컬럼으로 df 업데이트 
london_lists = london_lists[condition_review]
london_url = london_url[condition_review]

# print(london_lists[:10])
# print()
print(london_lists.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65869 entries, 0 to 90735
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        65869 non-null  int64  
 1   호스트_id       65869 non-null  int64  
 2   슈퍼호스트        65538 non-null  object 
 3   숙소_지역        65869 non-null  object 
 4   숙소_유형        65869 non-null  object 
 5   수용_인원수       65869 non-null  int64  
 6   욕실수          43758 non-null  float64
 7   침실수          58052 non-null  float64
 8   침대수          43703 non-null  float64
 9   편의시설         65869 non-null  object 
 10  숙소_가격        43728 non-null  object 
 11  리뷰수          65869 non-null  int64  
 12  30일_리뷰수      65869 non-null  int64  
 13  리뷰점수         65869 non-null  float64
 14  숙소_정확성_리뷰점수  65838 non-null  float64
 15  숙소_청결도_리뷰점수  65843 non-null  float64
 16  숙소_체크인_리뷰점수  65808 non-null  float64
 17  숙소_소통_리뷰점수   65833 non-null  float64
 18  숙소_위치_리뷰점수   65808 non-null  float64
 19  숙소_가

# 결측치 처리

In [199]:
# 컬럼 전체 결측치 확인 

london_lists.isnull().sum()
# 전체 개수 : 65,869개 중 

숙소_id              0
호스트_id             0
슈퍼호스트            331
숙소_지역              0
숙소_유형              0
수용_인원수             0
욕실수            22111
침실수             7817
침대수            22166
편의시설               0
숙소_가격          22141
리뷰수                0
30일_리뷰수            0
리뷰점수               0
숙소_정확성_리뷰점수       31
숙소_청결도_리뷰점수       26
숙소_체크인_리뷰점수       61
숙소_소통_리뷰점수        36
숙소_위치_리뷰점수        61
숙소_가격_리뷰점수        63
평균_리뷰수             0
dtype: int64

## 가격이 null값인 컬럼과 가격이 null값이 아닌 컬럼으로 분리 
* 가격과 기타 컬럼 간의 관계 분석 목적 

In [200]:
# 가격이 null값/notnull값 조건 생성 
condition_price_notnull = london_lists['숙소_가격'].notnull()
condition_price_null = london_lists['숙소_가격'].isnull()

# 가격이 null값/null값이 아닌 df 분리
london_lists_price = london_lists[condition_price_notnull] 
london_lists_price_null = london_lists[condition_price_null]

# df 데이터 개수 확인 
print(london_lists_price.shape)
print(london_lists_price_null.shape)

# 변수명 정리 
london_lists = london_lists_price

(43728, 21)
(22141, 21)


In [201]:
london_lists.isnull().sum()

숙소_id            0
호스트_id           0
슈퍼호스트          266
숙소_지역            0
숙소_유형            0
수용_인원수           0
욕실수             21
침실수             11
침대수             76
편의시설             0
숙소_가격            0
리뷰수              0
30일_리뷰수          0
리뷰점수             0
숙소_정확성_리뷰점수     17
숙소_청결도_리뷰점수     18
숙소_체크인_리뷰점수     18
숙소_소통_리뷰점수      17
숙소_위치_리뷰점수      18
숙소_가격_리뷰점수      19
평균_리뷰수           0
dtype: int64

## 슈퍼호스트 결측치 처리
* 우선 결측치 제거하고 진행

컬럼 전처리 상황
* 리뷰수가 0인 컬럼 제거
* 가격이 null값인 컬럼 제거한 상태 

슈퍼호스트 기준
1. 숙박 10건 이상 호스팅 또는 3건의 예약에 걸쳐 총 100박 이상 호스팅
2. 응답률 90% 이상 유지
3. 예약 취소율 1% 미만 유지. 단, 정상참작이 가능한 상황에 따른 예약 취소는 제외
4. 전체 평점 4.8점 이상 유지(후기 작성 기한인 14일이 지나거나, 그전에라도 게스트와 호스트 양측이 모두 후기를 제출하면 후기는 슈퍼호스트 실적에 계산됩니다).

In [202]:
# 슈퍼호스트 null 값 : 266개
print(f'슈퍼호스트 null값 개수: {london_lists["슈퍼호스트"].isnull().sum()}개')

# 슈퍼호스트 null값의 비율: 0.61% >> 삭제해도 무방 
print(f'슈퍼호스트 null값의 비율: {round((london_lists["슈퍼호스트"].isnull().sum() / london_lists.shape[0]) * 100, 2)}%')

슈퍼호스트 null값 개수: 266개
슈퍼호스트 null값의 비율: 0.61%


In [203]:
# 리뷰가 0개인 숙소를 모아놓은 데이터프레임에도 적용 

# 슈퍼호스트 null 값 : 58개
print(f'슈퍼호스트 null값 개수: {london_lists_review_0["슈퍼호스트"].isnull().sum()}개')

# 슈퍼호스트 null값의 비율: 0.23% >> 삭제해도 무방 
print(f'슈퍼호스트 null값의 비율: {round((london_lists_review_0["슈퍼호스트"].isnull().sum() / london_lists_review_0.shape[0]) * 100, 2)}%')

슈퍼호스트 null값 개수: 58개
슈퍼호스트 null값의 비율: 0.23%


In [204]:
london_lists['슈퍼호스트'].value_counts()

f    32130
t    11332
Name: 슈퍼호스트, dtype: int64

슈퍼호스트 컬럼 / 전처리 방향 설정 결론 (슈퍼호스트_런던_240524.ipynb 파일 참고)

- null 값을 삭제해도 무방하다. (비율이 0.6%)
- 결측치를 일괄 대치하려면 최빈값으로 대치하는 것이 맞을듯
- 결측치를 실제 데이터와 비교해보는 방법도 있다. 
    - NGD 기법 또는 크롤링 

In [205]:
# 슈퍼호스트 결측치 제거
print(f'삭제 전 전체 데이터 개수: {london_lists.shape[0]}개')
print(f"삭제 할 슈퍼호스트 null값 데이터 개수: {london_lists['슈퍼호스트'].isnull().sum()}개")
london_lists = london_lists.dropna(subset=['슈퍼호스트'])
print()

# 제거 후 확인 
print(f"제거 후 슈퍼호스트 컬럼의 null값 확인: {london_lists['슈퍼호스트'].isnull().sum()}개")
print(f'Null값 제거 후 전체 데이터 개수 : {london_lists.shape[0]}개')


삭제 전 전체 데이터 개수: 43728개
삭제 할 슈퍼호스트 null값 데이터 개수: 266개

제거 후 슈퍼호스트 컬럼의 null값 확인: 0개
Null값 제거 후 전체 데이터 개수 : 43462개


## 욕실수 / 침실수 / 침대수 결측치 처리 
* 결론: 우선 결측치 제거하고 진행 

현재 데이터 전처리 현황 
* 리뷰수가 0개인 컬럼 제거 
* 가격이 Null값이 아닌 컬럼만 선택 
* 슈퍼호스트가 Null값인 데이터 제거 

In [206]:
# 결측치 수치 확인 
london_lists.isnull().sum()

숙소_id           0
호스트_id          0
슈퍼호스트           0
숙소_지역           0
숙소_유형           0
수용_인원수          0
욕실수            21
침실수            11
침대수            76
편의시설            0
숙소_가격           0
리뷰수             0
30일_리뷰수         0
리뷰점수            0
숙소_정확성_리뷰점수    17
숙소_청결도_리뷰점수    18
숙소_체크인_리뷰점수    18
숙소_소통_리뷰점수     17
숙소_위치_리뷰점수     18
숙소_가격_리뷰점수     19
평균_리뷰수          0
dtype: int64

#### 결측치 확인

In [207]:
# 욕실수, 침실수, 침대수가 공통적으로 Null값인 컬럼 확인 

facility_null_list = [['욕실수', '침실수', '침대수'], ['욕실수', '침대수'], ['욕실수', '침실수'], ['침대수', '침실수']]

for facility in facility_null_list:
    condition_facilities_null = london_lists[facility].isnull().all(axis=1)
    print(f'{facility}가 모두 null값인 데이터의 개수: {london_lists[condition_facilities_null].shape[0]}개')

['욕실수', '침실수', '침대수']가 모두 null값인 데이터의 개수: 0개
['욕실수', '침대수']가 모두 null값인 데이터의 개수: 1개
['욕실수', '침실수']가 모두 null값인 데이터의 개수: 0개
['침대수', '침실수']가 모두 null값인 데이터의 개수: 0개


In [208]:
# 전체 데이터 대비 ('욕실수', '침실수', '침대수' 컬럼의 결측치 합 개수) 비율 확인 : 0.25%

facilities = ['욕실수', '침실수', '침대수']

sum_facilities_null = 0

for f in facilities:
    sum = london_lists[f].isnull().sum()
    sum_facilities_null += sum
    
ratio_facilities_null = round(((sum_facilities_null - london_lists[["욕실수", "침대수"]].isnull().all(axis=1).sum()) / london_lists.shape[0]) * 100, 2)
print(f'욕실수, 침대수, 침실수의 null값 합계 : {(sum_facilities_null - london_lists[["욕실수", "침대수"]].isnull().all(axis=1).sum())}개')
print(f'전체 데이터 대비 숙소 시설 결측치 비율 : {ratio_facilities_null}%')

욕실수, 침대수, 침실수의 null값 합계 : 107개
전체 데이터 대비 숙소 시설 결측치 비율 : 0.25%


#### 제거 가능 하지만 크롤링한 데이터 비교해보자 
* 침대수, 침실수, 욕실수 데이터는 시간의 영향을 매우 적은 데이터 

In [209]:
fac1 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_1.csv', encoding='cp949')
fac2 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_2.csv', encoding='cp949')
fac3 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_3.csv', encoding='cp949')
fac4 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_4.csv', encoding='cp949')

crawl_fac_null = pd.concat([fac1, fac2, fac3, fac4], axis=0, ignore_index=True)
print(crawl_fac_null.shape)
print(crawl_fac_null.info())

(4000, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호스트_id  1000 non-null   float64
 1   숙소_url  4000 non-null   object 
 2   etc     3208 non-null   object 
 3   숙소_id   3000 non-null   float64
dtypes: float64(2), object(2)
memory usage: 125.1+ KB
None


In [210]:
crawl_fac_null[:10]

,호스트_id,숙소_url,etc,숙소_id
0,170702.0,https://www.airbnb.com/rooms/170702,침대 1개 · 공용 욕실,NaN
1,314985.0,https://www.airbnb.com/rooms/314985,침대 1개 · 욕실 없음,NaN
2,315658.0,https://www.airbnb.com/rooms/315658,더블 침대 1개 · 공용 욕실,NaN
3,39387.0,https://www.airbnb.com/rooms/39387,소파 베드 1개 · 공용 욕실,NaN
4,41509.0,https://www.airbnb.com/rooms/41509,침대 3개 · 공용 욕실,NaN
5,46992.0,https://www.airbnb.com/rooms/46992,퀸사이즈 침대 1개 · 침실에 딸린 전용 욕실,NaN
6,330044.0,https://www.airbnb.com/rooms/330044,퀸사이즈 침대 1개 · 전용 욕실,NaN
7,229684.0,https://www.airbnb.com/rooms/229684,침대 1개 · 공용 욕실,NaN
8,72913.0,https://www.airbnb.com/rooms/72913,더블 침대 1개 · 침실에 딸린 전용 욕실,NaN
9,341143.0,https://www.airbnb.com/rooms/341143,침대 1개 · 공용 욕실,NaN


당장 사용하긴 어려울 듯. 일단 제거하고 추후에 다시 집어넣자

#### 결측치 제거

In [211]:
# 결측치 제거 
print(f'삭제 전 전체 데이터 개수: {london_lists.shape[0]}개')
print(f"""삭제 할 슈퍼호스트 null값 데이터 개수: 
{london_lists[['욕실수', '침실수', '침대수']].isnull().sum()}""")
london_lists = london_lists.dropna(subset=['욕실수', '침실수', '침대수'])
print()

# 제거 완료됐나 확인
print(f"""남아있는 욕실수, 침실수, 침대수 null값 데이터 개수:  
{london_lists[['욕실수', '침실수', '침대수']].isnull().sum()}""")
print(f'Null값 제거 후 전체 데이터 개수: {london_lists.shape[0]}개')

삭제 전 전체 데이터 개수: 43462개
삭제 할 슈퍼호스트 null값 데이터 개수: 
욕실수    21
침실수    11
침대수    76
dtype: int64

남아있는 욕실수, 침실수, 침대수 null값 데이터 개수:  
욕실수    0
침실수    0
침대수    0
dtype: int64
Null값 제거 후 전체 데이터 개수: 43355개


## 리뷰 점수 컬럼 결측치 처리 


현재 데이터 전처리 현황 
* 리뷰수가 0개인 컬럼 제거 
* 가격이 Null값이 아닌 컬럼만 선택 
* 슈퍼호스트가 Null값인 데이터 제거 
* 욕실수 / 침실수 / 침대수 결측치 제거

In [213]:
# 결측치 확인
london_lists.isnull().sum()

숙소_id           0
호스트_id          0
슈퍼호스트           0
숙소_지역           0
숙소_유형           0
수용_인원수          0
욕실수             0
침실수             0
침대수             0
편의시설            0
숙소_가격           0
리뷰수             0
30일_리뷰수         0
리뷰점수            0
숙소_정확성_리뷰점수    16
숙소_청결도_리뷰점수    17
숙소_체크인_리뷰점수    17
숙소_소통_리뷰점수     16
숙소_위치_리뷰점수     17
숙소_가격_리뷰점수     18
평균_리뷰수          0
dtype: int64

In [222]:
# 각 리뷰 점수 컬럼이 모두 NULL 값인 데이터 개수 확인 : 15개
type_review_null_all = london_lists[['숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']].isnull().all(axis=1)
print(f'각 기준별 리뷰점수가 모두 Null값인 데이터 개수: {london_lists[type_review_null_all].shape[0]}개')
print()

# 데이터 비율 확인 : 0.03%
print(f"전체 데이터 개수: {london_lists.shape[0]}개")
print(f"각 기준별 리뷰점수가 모두 NULL값인 데이터의 비율: {round((london_lists[type_review_null_all].shape[0] / london_lists.shape[0]) * 100, 2)}%")
print()

# 모두 NULL값인 데이터 제거
london_lists = london_lists[~type_review_null_all]
print(f"NULL값 제거 후 전체 데이터 개수: {london_lists.shape[0]}개")

각 기준별 리뷰점수가 모두 Null값인 데이터 개수: 15개

전체 데이터 개수: 43355개
각 기준별 리뷰점수가 모두 NULL값인 데이터의 비율: 0.03%
NULL값 제거 후 전체 데이터 개수: 43340개


In [229]:
# 각 리뷰 점수 컬럼별로 하나라도 NULL 값을 가지고 있는 데이터 개수 확인: 7개
type_review_null_any = london_lists[['숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']].isnull().any(axis=1)
print(f'각 기준별 리뷰점수가 하나라도 Null값인 데이터 개수: {london_lists[type_review_null_any].shape[0]}개')
print()

# 데이터 비율 확인 : 0.02%
print(f"제거 전 전체 데이터 개수: {london_lists.shape[0]}개")
print(f"각 기준별 리뷰점수 중 하나라도 NULL값인 데이터의 비율: {round((london_lists[type_review_null_any].shape[0] / london_lists.shape[0]) * 100, 2)}%")
print()

# 하나라도 NULL값인 데이터 제거
london_lists = london_lists[~type_review_null_any]
print(f"NULL값 제거 후 전체 데이터 개수: {london_lists.shape[0]}개")

각 기준별 리뷰점수가 하나라도 Null값인 데이터 개수: 7개

제거 전 전체 데이터 개수: 43340개
각 기준별 리뷰점수 중 하나라도 NULL값인 데이터의 비율: 0.02%

NULL값 제거 후 전체 데이터 개수: 43333개


## 결측치 처리 후 데이터 확인 

현재 데이터 전처리 현황 
* 리뷰수가 0개인 컬럼 제거 
* 가격이 Null값이 아닌 컬럼만 선택 
* 슈퍼호스트가 Null값인 데이터 제거 
* 욕실수 / 침실수 / 침대수 결측치 제거
* 각 리뷰 점수 컬럼 결측치 제거 

In [230]:
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43333 entries, 0 to 90735
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        43333 non-null  int64  
 1   호스트_id       43333 non-null  int64  
 2   슈퍼호스트        43333 non-null  object 
 3   숙소_지역        43333 non-null  object 
 4   숙소_유형        43333 non-null  object 
 5   수용_인원수       43333 non-null  int64  
 6   욕실수          43333 non-null  float64
 7   침실수          43333 non-null  float64
 8   침대수          43333 non-null  float64
 9   편의시설         43333 non-null  object 
 10  숙소_가격        43333 non-null  object 
 11  리뷰수          43333 non-null  int64  
 12  30일_리뷰수      43333 non-null  int64  
 13  리뷰점수         43333 non-null  float64
 14  숙소_정확성_리뷰점수  43333 non-null  float64
 15  숙소_청결도_리뷰점수  43333 non-null  float64
 16  숙소_체크인_리뷰점수  43333 non-null  float64
 17  숙소_소통_리뷰점수   43333 non-null  float64
 18  숙소_위치_리뷰점수   43333 non-null  float64
 19  숙소_가

In [234]:
london_lists['수용_인원수'].unique()

array([ 2,  1,  6,  4,  5,  3,  7,  8, 12,  9, 10, 13, 15, 11, 16, 14])

In [233]:
london_lists['수용_인원수'].value_counts().sort_index()

1      4869
2     16581
3      3446
4      9242
5      2582
6      3815
7       782
8      1120
9       254
10      329
11       46
12      114
13       30
14       36
15       17
16       70
Name: 수용_인원수, dtype: int64

수용 가능 인원수로도 숙소의 크기를 가늠할 수 있지 않을까?